In [1]:
from scdeepinsight import pbmc
from timeit import default_timer as timer

In [2]:
start = timer()
#ImageTransform(query_path:str, barcode_path:str, image_path:str)
#query_path: The absoulte path of the target scRNA-seq dataset (end with .h5ad).
#barcode_path: The absoulte path where you wish to store the generated barcode file (end with .csv). 
#image_path: The absoulte path where you wish to store the generated image file (end with .npy).
pbmc.ImageTransform("sample.h5ad", "barcode.csv", "image.npy")

/home/shangru/miniconda3/envs/sr/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


In [3]:
#Annotate (barcode_path:str, image_path:str, batch_size:int = 128)
#barcode_path: The absoulte path where you have stored the generated barcode file (end with .csv). 
#image_path: The absoulte path where you have stored the generated image file (end with .npy). 
#batch_size: The batch size you wish to load to the converted image dataset when using the pretrained model for annotation. Defaulted value is 128.
pred_label = pbmc.Annotate("barcode.csv", "image.npy", batch_size=128)
pred_label

Loaded pretrained weights for efficientnet-b3


,0
TGCGCAGCATCAGTAC-19,B memory
AAGGTTCCAACCGCCA-28,B naive
CTACCCACAGATGGGT-15,cDC2
ATAAGAGCACTAGTAC-26,B naive
TTATGCTCAATGCCAT-2,NK
...,...
TCTTTCCAGTGAAGAG-10,B memory
AGTCTTTCAGCTGTTA-28,NK
TTCTTAGTCTGCCCTA-17,NK
GGACATTAGACGCTTT-16,CD4 Naive


In [4]:
end = timer()
print("Total time cost in the annotation process: %.03f usec" % (end - start))

Total time cost in the annotation process: 55.358 usec


In [5]:
import pandas as pd
# Use the provided real cell types to validate the annotation result
real_label = pd.read_csv("real_sample_labels.csv", index_col=0)
real_label

,celltype
barcode,
TGCGCAGCATCAGTAC-19,B memory
AAGGTTCCAACCGCCA-28,B naive
CTACCCACAGATGGGT-15,cDC2
ATAAGAGCACTAGTAC-26,B naive
TTATGCTCAATGCCAT-2,NK
...,...
TCTTTCCAGTGAAGAG-10,B memory
AGTCTTTCAGCTGTTA-28,NK
TTCTTAGTCTGCCCTA-17,NK


In [6]:
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
ls = list(set(pd.unique(pred_label[0]).tolist()) | set(pd.unique(real_label["celltype"]).tolist()))
acc = accuracy_score(pred_label,real_label)
le2 = preprocessing.LabelEncoder()
le2.fit(ls)
pred_label_2 = le2.transform(pred_label[0])
real_label_2 = le2.transform(real_label["celltype"])
acc = accuracy_score(pred_label,real_label)
ari = adjusted_rand_score(real_label_2, pred_label_2)
print("Accuray: %.03f, ARI = %.03f" % (acc, ari))

Accuray: 0.947, ARI = 0.958
